# Get point cloud from Waymo v2 format

In [2]:
#@title Initial setup
from typing import Optional
import warnings
# Disable annoying warnings from PyArrow using under the hood.
warnings.simplefilter(action='ignore', category=FutureWarning)
from platform import python_version
print(python_version())
import numpy as np


import tensorflow as tf
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
import dask.dataframe as dd
from waymo_open_dataset import v2
from waymo_open_dataset.utils import  frame_utils


# Path to the directory with all components
dataset_dir = '/home/wod_v2/src/data/perception_2.0.0/training'

context_name = '10023947602400723454_1120_000_1140_000'

def read(tag: str) -> dd.DataFrame:
  """Creates a Dask DataFrame for the component specified by its tag."""
  paths = tf.io.gfile.glob(f'{dataset_dir}/{tag}/{context_name}.parquet')
  return dd.read_parquet(paths)



# Lazily read DataFrames for all components.
association_df = read('camera_to_lidar_box_association')
cam_box_df = read('camera_box')
cam_img_df = read('camera_image')
lidar_box_df = read('lidar_box')
lidar_df = read('lidar')
lidar_calibration_df = read('lidar_calibration')

association_df = association_df[association_df['key.camera_name'] == 1]
cam_box_df = cam_box_df[cam_box_df['key.camera_name'] == 1]
cam_img_df = cam_img_df[cam_img_df['key.camera_name'] == 1].compute()
lidar_df = lidar_df[lidar_df['key.laser_name'] == 1]
lidar_calibration_df = lidar_calibration_df[lidar_calibration_df['key.laser_name'] == 1].compute()

# Join all DataFrames using matching columns
cam_image_w_box_df = v2.merge(cam_box_df, cam_img_df)
cam_obj_df = v2.merge(association_df, cam_image_w_box_df)
# In this example camera box labels are optional, so we set left_nullable=True.
obj_df = v2.merge(cam_obj_df, lidar_box_df, left_nullable=True)
# Group lidar sensors (left), group labels and camera images (right) and join.
df = v2.merge(lidar_df, obj_df, left_group=True, right_group=True)

3.10.14


2024-08-15 03:11:51.787328: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.12.0
Num GPUs Available:  1


2024-08-15 03:11:55.260555: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-15 03:11:55.285392: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-15 03:11:55.288371: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
# Read a single row, which contain data for all data for a single frame.
_, row = next(iter(df.iterrows()))
# Create all component objects
camera_image = v2.CameraImageComponent.from_dict(row)
lidar = v2.LiDARComponent.from_dict(row)
camera_box = v2.CameraBoxComponent.from_dict(row)
lidar_box = v2.LiDARBoxComponent.from_dict(row)
lidar_calibration = v2.LiDARCalibrationComponent.from_dict(lidar_calibration_df)

print(
    f'Found {len(lidar_box.key.laser_object_id)} objects on'
    f' {lidar.key.segment_context_name=} {lidar.key.frame_timestamp_micros=}'
)
for laser_object_id, camera_object_id, camera_name in zip(
    lidar_box.key.laser_object_id,
    camera_box.key.camera_object_id,
    camera_image.key.camera_name,
):
  print(f'\t{laser_object_id=} {camera_object_id=} {camera_name=}')

Found 78 objects on lidar.key.segment_context_name='10023947602400723454_1120_000_1140_000' lidar.key.frame_timestamp_micros=1552440195362591
	laser_object_id='-U88NMYnocLWCh6iqZwj1g' camera_object_id=nan camera_name=nan
	laser_object_id='0VCoeT-jjrIfzTCsOWz20A' camera_object_id=nan camera_name=nan
	laser_object_id='0_HBXNo3olLueqYvkPohlg' camera_object_id=nan camera_name=nan
	laser_object_id='1nDCER_bA9py1ZPpNXecog' camera_object_id=nan camera_name=nan
	laser_object_id='2-A6zakvKX2opVnyx9gplQ' camera_object_id='a6f937a6-7ea8-4393-b636-e0560e699856' camera_name=1.0
	laser_object_id='2OYKagQRfCdaOXgU5RkMBA' camera_object_id=nan camera_name=nan
	laser_object_id='2SYmRAjI0pCOwp2XYemMBQ' camera_object_id='ba670814-995e-4ade-bc42-58b0a1d8ec8d' camera_name=1.0
	laser_object_id='3083QteOhZ_vSpxmP0XK-Q' camera_object_id=nan camera_name=nan
	laser_object_id='38Np8bwqcvw9KkrH3xHfpg' camera_object_id=nan camera_name=nan
	laser_object_id='3I9vmrICjjnWtavq3ysJrQ' camera_object_id=nan camera_name=na

In [5]:
# Get Range image of top lidar
range_image_orig = lidar.range_image_return1

range_image = v2.perception.lidar.RangeImage(lidar.range_image_return1.values[0], lidar.range_image_return1.shape[0])

print(range_image)
print(type(range_image.values))
print(range_image.shape)

temp_tfm = v2.column_types.Transform
temp_tfm.transform = lidar_calibration.extrinsic.transform.tolist()[0]
temp_bic = v2.perception.context.BeamInclination
temp_bic.min = lidar_calibration.beam_inclination.min
temp_bic.max = lidar_calibration.beam_inclination.max
temp_bic.values = lidar_calibration.beam_inclination.values.tolist()[0]

lc2 = v2.perception.context.LiDARCalibrationComponent(lidar_calibration.key, temp_tfm, temp_bic)

print(lidar_calibration.extrinsic.transform.tolist()[0])
print(type(lidar_calibration.extrinsic))
print(type(lidar_calibration.extrinsic.transform))
print(type(lidar_calibration_df))
print(type(lidar_calibration))
# extrinsic = tf.convert_to_tensor(lidar_calibration.extrinsic.transform)

# TODO: Eventually should add pixel_pose and frame_pose when mulitple cameras are used
points = v2.perception.utils.lidar_utils.convert_range_image_to_point_cloud(range_image, lc2, keep_polar_features=True)


RangeImage(values=array([-1., -1., -1., ..., -1., -1., -1.], dtype=float32), shape=array([  64, 2650,    4], dtype=int32))
<class 'numpy.ndarray'>
[  64 2650    4]
[-8.50486534e-01 -5.25981865e-01 -3.96644352e-03  1.43000000e+00
  5.25971210e-01 -8.50495357e-01  3.45458508e-03  0.00000000e+00
 -5.19049090e-03  8.51842992e-04  9.99986166e-01  2.18400000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]
<class 'waymo_open_dataset.v2.column_types.Transform'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
<class 'waymo_open_dataset.v2.perception.context.LiDARCalibrationComponent'>


In [6]:
print(range_image.shape)
ri = np.reshape(range_image.values, range_image.shape)
while (1):
    cv2.imshow('range_image', ri)
    if cv2.waitKey(10) & 0xFF == 27:
        break
cv2.destroyAllWindows

img = np.array(camera_image.image[4])
print(len(camera_image.image[4]))
while (1):
    cv2.imshow('image', img)
    if cv2.waitKey(10) & 0xFF == 27:
        break
cv2.destroyAllWindows

[  64 2650    4]
387998


error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'imshow'
> Overload resolution failed:
>  - mat data type = |S387998 is not supported
>  - Expected Ptr<cv::cuda::GpuMat> for argument 'mat'
>  - Expected Ptr<cv::UMat> for argument 'mat'


In [4]:
print(points.shape)
print(points[:,4])

(149796, 6)
tf.Tensor([ 4.752197   4.3479753  8.619505  ... -1.8297869 -1.8156391 -1.7855257], shape=(149796,), dtype=float32)


# Pipeline to yolo bev format

In [6]:
import cv2
import numpy as np


# create birds-eye view of lidar data
def bev_from_pcl(lidar_pcl, configs, viz=False, verbose=False):
    # remove lidar points outside detection area and with too low reflectivity
    mask = np.where((lidar_pcl[:, POINTCLOUD_X_INDEX] >= configs['lim_x'][0]) & (lidar_pcl[:, POINTCLOUD_X_INDEX] <= configs['lim_x'][1]) &
                    (lidar_pcl[:, POINTCLOUD_Y_INDEX] >= configs['lim_y'][0]) & (lidar_pcl[:, POINTCLOUD_Y_INDEX] <= configs['lim_y'][1]))# &
                    #(lidar_pcl[:, POINTCLOUD_Z_INDEX] >= configs['lim_z'][0]) & (lidar_pcl[:, POINTCLOUD_Z_INDEX] <= configs['lim_z'][1]))
    lidar_pcl = lidar_pcl[mask]
    
    # shift level of ground plane to avoid flipping from 0 to 255 for neighboring pixels
    lidar_pcl[:, POINTCLOUD_Z_INDEX] = lidar_pcl[:, POINTCLOUD_Z_INDEX] - configs['lim_z'][0]  

    if verbose:
        print(lidar_pcl[0,:])
        print('Min and max height, %f, %f' %(np.min(lidar_pcl[:,2]), np.max(lidar_pcl[:,2])))

    # convert sensor coordinates to bev-map coordinates (center is bottom-middle)

    ## step 1 : compute bev-map discretization by dividing x-range by the bev-image height (see configs)
    delta_x_rw_meters = configs['lim_x'][1] - configs['lim_x'][0]
    delta_y_rw_meters = configs['lim_y'][1] - configs['lim_y'][0]
    meters_pixel_x = delta_x_rw_meters / configs['bev_height']
    meters_pixel_y = delta_y_rw_meters / configs['bev_width']

    ## step 2 : create a copy of the lidar pcl and transform all metrix x-coordinates into bev-image coordinates  
    lidar_pcl_copy = np.copy(lidar_pcl)
    lidar_pcl_copy[:, POINTCLOUD_X_INDEX] = np.int_(np.floor(lidar_pcl_copy[:, POINTCLOUD_X_INDEX] / meters_pixel_x))  

    # step 3 : perform the same operation as in step 2 for the y-coordinates but make sure that no negative bev-coordinates occur
    lidar_pcl_copy[:, POINTCLOUD_Y_INDEX] = np.int_(np.floor(lidar_pcl_copy[:, POINTCLOUD_Y_INDEX] / meters_pixel_y) + (configs['bev_width'] + 1) / 2)

    # step 4 : visualize point-cloud using the function show_pcl from a previous task
    # if viz:
    #     show_pcl(lidar_pcl_copy)
   
   
    # Compute intensity layer of the BEV map

    ## step 1 : create a numpy array filled with zeros which has the same dimensions as the BEV map
    intensity_map = np.zeros((configs['bev_height'] + 1, configs['bev_width'] + 1))

    # step 2 : re-arrange elements in lidar_pcl_cpy by sorting first by x, then y, then -z (use numpy.lexsort)
    lidar_pcl_copy[lidar_pcl_copy[:, POINTCLOUD_INTENSITY] > 1.0, POINTCLOUD_INTENSITY] = 1.0
    index_vector_int = np.lexsort((-lidar_pcl_copy[:, POINTCLOUD_INTENSITY], lidar_pcl_copy[:, POINTCLOUD_Y_INDEX], lidar_pcl_copy[:, POINTCLOUD_X_INDEX]))
    lidar_pcl_top = lidar_pcl_copy[index_vector_int]

    ## step 3 : extract all points with identical x and y such that only the top-most z-coordinate is kept (use numpy.unique)
    ##          also, store the number of points per x,y-cell in a variable named "counts" for use in the next task
    _, idx_int_unique, counts = np.unique(lidar_pcl_top[:, POINTCLOUD_X_INDEX:POINTCLOUD_Z_INDEX], return_index=True, return_inverse=False, return_counts=True, axis=0)
    lidar_pcl_top = lidar_pcl_top[idx_int_unique]

    intensity_map = np.zeros((configs['bev_height'] + 1, configs['bev_width'] + 1))
    
    ## step 4 : assign the intensity value of each unique entry in lidar_pcl_top to the intensity map 
    ##          make sure that the intensity is scaled in such a way that objects of interest (e.g. vehicles) are clearly visible    
    ##          also, make sure that the influence of outliers is mitigated by normalizing intensity on the difference between the max. and min. value within the point cloud
    intensity_map[np.int_(lidar_pcl_top[:, POINTCLOUD_X_INDEX]), np.int_(lidar_pcl_top[:, POINTCLOUD_Y_INDEX])] = lidar_pcl_top[:, POINTCLOUD_INTENSITY] / (np.amax(lidar_pcl_top[:, POINTCLOUD_INTENSITY]) - np.amin(lidar_pcl_top[:, POINTCLOUD_INTENSITY]))

    # if viz:
    #     analyze({'before': lidar_pcl_top_copy, 'after': lidar_pcl_top_copy_post}, title='Intensity Distribution', nqp=False)

    ## step 5 : temporarily visualize the intensity map using OpenCV to make sure that vehicles separate well from the background
    img_intensity = intensity_map * 255
    img_intensity = img_intensity.astype(np.uint8)
    if viz:
        while (1):
            cv2.imshow('img_intensity', img_intensity)
            if cv2.waitKey(10) & 0xFF == 27:
                break
        cv2.destroyAllWindows

   

    # Compute height layer of the BEV map

    ## step 1 : create a numpy array filled with zeros which has the same dimensions as the BEV map
    height_map = np.zeros((configs['bev_height'] + 1, configs['bev_width'] + 1))

    ## step 2 : assign the height value of each unique entry in lidar_top_pcl to the height map
    ##          make sure that each entry is normalized on the difference between the upper and lower height defined in the config file
    ##          use the lidar_pcl_top data structure from the previous task to access the pixels of the height_map
    # _, idx_height_unique, counts = np.unique(lidar_pcl_top[:, 0:2], return_index=True, return_inverse=False, return_counts=True, axis=0)
    # lidar_pcl_hei = lidar_pcl_top[idx_height_unique]
    height_map[np.int_(lidar_pcl_top[:, POINTCLOUD_X_INDEX]), np.int_(lidar_pcl_top[:, POINTCLOUD_Y_INDEX])] = lidar_pcl_top[:, POINTCLOUD_Z_INDEX] / float(np.abs(configs['lim_z'][1] - configs['lim_z'][0]))

    ## step 3 : temporarily visualize the intensity map using OpenCV to make sure that vehicles separate well from the background
    img_height = height_map * 256
    img_height = img_height.astype(np.uint8)
    if viz:
        while (1):
            cv2.imshow('img_height', img_height)
            if cv2.waitKey(10) & 0xFF == 27:
                break
        cv2.destroyAllWindows

    #######
    ####### ID_S2_EX3 END #######       

    # TODO remove after implementing all of the above steps
    # lidar_pcl_cpy = []
    # lidar_pcl_top = []
    # height_map = []
    # intensity_map = []

    # Compute density layer of the BEV map


    density_map = np.zeros((configs['bev_height'] + 1, configs['bev_width'] + 1))
    _, _, counts = np.unique(lidar_pcl_copy[:, POINTCLOUD_X_INDEX:POINTCLOUD_Z_INDEX], axis=0, return_index=True, return_counts=True)
    normalizedCounts = np.minimum(1.0, np.log(counts + 1) / np.log(64)) 
    density_map[np.int_(lidar_pcl_top[:, POINTCLOUD_X_INDEX]), np.int_(lidar_pcl_top[:, POINTCLOUD_Y_INDEX])] = normalizedCounts

    
    if viz:
        while (1):
            cv2.imshow('density map', density_map)
            if cv2.waitKey(10) & 0xFF == 27:
                break
        cv2.destroyAllWindows
    
    # assemble 3-channel bev-map from individual maps
    bev_map = np.zeros((3, configs['bev_height'], configs['bev_width']))
    bev_map[2, :, :] = density_map[:configs['bev_height'], :configs['bev_width']]  # r_map
    bev_map[1, :, :] = height_map[:configs['bev_height'], :configs['bev_width']]  # g_map
    bev_map[0, :, :] = intensity_map[:configs['bev_height'], :configs['bev_width']]  # b_map
    
    bev_map_cpy = np.zeros((configs['bev_height'], configs['bev_width'], 3))
    bev_map_cpy[:, :, 2] = density_map[:configs['bev_height'], :configs['bev_width']] /  density_map.max() # r_map
    bev_map_cpy[:, :, 1] = height_map[:configs['bev_height'], :configs['bev_width']] / height_map.max() # g_map
    bev_map_cpy[:, :, 0] = intensity_map[:configs['bev_height'], :configs['bev_width']] / intensity_map.max() # b_map

    # expand dimension of bev_map before converting into a tensor
    s1, s2, s3 = bev_map.shape
    bev_maps = np.zeros((1, s1, s2, s3))
    bev_maps[0] = bev_map

    if viz:
        print(bev_map_cpy.shape)
        # bev_map_cpy = np.reshape(bev_map, (bev_map.shape[1], bev_map.shape[2], bev_map.shape[0]))
        print( intensity_map.max(), height_map.max(), density_map.max())
        while (1):
            cv2.imshow('bev_map', bev_map_cpy)
            if cv2.waitKey(10) & 0xFF == 27:
                break
        cv2.destroyAllWindows
    # bev_maps = torch.from_numpy(bev_maps)  # create tensor from birds-eye view
    # input_bev_maps = bev_maps.to(configs[device], non_blocking=True).float()

    # show_bev(input_bev_maps, configs)

    return bev_maps

In [7]:
configs = {}
configs['lim_x'] = [0,53]
configs['lim_y'] = [-20,20]
configs['lim_z'] = [-3.1, 2]
configs['lim_r'] = [0, 1.0]
configs['bev_height'] = 640
configs['bev_width'] = 480

# points_xyzl = points[:, :4]
# points_xyzl[:, :2] = points[:, 3:5]
# points_xyzl[:, 3] = points[:, 1]
POINTCLOUD_X_INDEX = 3
POINTCLOUD_Y_INDEX = 4
POINTCLOUD_Z_INDEX = 5
POINTCLOUD_INTENSITY = 1
# print(points_xyzl[:, POINTCLOUD_Z_INDEX].min(), ' ', points_xyzl[:, POINTCLOUD_Z_INDEX].max())

points_cpu = points.numpy()
bev_map = bev_from_pcl(points_cpu, configs, viz=False)

### Pipeline for Yolov8

In [12]:
from ultralytics import YOLO

model = YOLO("yolov8n.yaml")
results = model.train(data="coco8.yaml")

Ultralytics YOLOv8.2.77 🚀 Python-3.10.14 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3080, 11941MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=coco8.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sho

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 433k/433k [00:00<00:00, 29.5MB/s]
Unzipping /home/wod_v2/src/datasets/coco8.zip to /home/wod_v2/src/datasets/coco8...: 100%|██████████| 25/25 [00:00<00:00, 5473.31file/s]

Dataset download success ✅ (0.3s), saved to /home/wod_v2/src/datasets




100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 755k/755k [00:00<00:00, 41.7MB/s]


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

...

                   all          4         17          0          0          0          0



100 epochs completed in 0.016 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.77 🚀 Python-3.10.14 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3080, 11941MiB)
YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 77.90it/s]

                   all          4         17          0          0          0          0


Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs/detect/train


## Visualizations

In [8]:
def rotate_mtx(points, angle, center):
    rotation = np.array([[np.cos(angle), -np.sin(angle)],[np.sin(angle), np.cos(angle)]])
    p = points - center
    p = p.dot(rotation)
    # print(p.shape)
    return p + center
    

def distance_to_pixel(box):
    w_ = configs['bev_height'] / (configs['lim_x'][1] - configs['lim_x'][0])
    l_ = configs['bev_width'] / (configs['lim_y'][1] - configs['lim_y'][0])
    x1 = (box[0] - box[2] / 2 - configs['lim_x'][0]) * w_
    x2 = (box[0] + box[2] / 2 - configs['lim_x'][0]) * w_
    y1 = (box[1] - box[3] / 2 - configs['lim_y'][0]) * l_
    y2 = (box[1] + box[3] / 2 - configs['lim_y'][0]) * l_
    # points = np.array([[x1,y1],[x2,y1],[x1,y2],[x2,y2]])
    return np.array([[y1,x1],[y1,x2],[y2,x2],[y2,x1]])
    # return rotate(points, box[4])
    
    # points = np.array([x1,y1,x2,y2]).astype(int)
    # return points

def distance_to_pixel2(box):
    w_ = configs['bev_height'] / (configs['lim_x'][1] - configs['lim_x'][0])
    l_ = configs['bev_width'] / (configs['lim_y'][1] - configs['lim_y'][0])
    x1 = (box[0] - box[2] / 2 - configs['lim_x'][0]) * w_
    x2 = (box[0] + box[2] / 2 - configs['lim_x'][0]) * w_
    y1 = (box[1] - box[3] / 2 - configs['lim_y'][0]) * l_
    y2 = (box[1] + box[3] / 2 - configs['lim_y'][0]) * l_
    # points = np.array([[x1,y1],[x2,y1],[x1,y2],[x2,y2]])
    # return np.array([[y1,x1],[y1,x2],[y2,x2],[y2,x1]])
    # return rotate(points, box[4])
    
    points = np.array([x1,y1,x2,y2]).astype(int)
    return points

In [ ]:
# Get box
print(type(lidar_box.box.center.x))

colors = [(0,100,100),(100,100,0), (100, 0, 100),(25,25,100)] #  Cyclists, Vehicles, Pedestrians, Signs

POINTCLOUD_X_INDEX = 0
POINTCLOUD_Y_INDEX = 1
lidar_box_masked = np.transpose(np.array([np.array(lidar_box.box.center.x), np.array(lidar_box.box.center.y), np.array(lidar_box.box.size.x), np.array(lidar_box.box.size.y), np.array(lidar_box.box.heading), np.array(lidar_box.type)]))
print(lidar_box_masked[0, :])
mask = np.where((lidar_box_masked[:, POINTCLOUD_X_INDEX] >= configs['lim_x'][0]) & (lidar_box_masked[:, POINTCLOUD_X_INDEX] <= configs['lim_x'][1]) &
                (lidar_box_masked[:, POINTCLOUD_Y_INDEX] >= configs['lim_y'][0]) & (lidar_box_masked[:, POINTCLOUD_Y_INDEX] <= configs['lim_y'][1]))# &
                #(lidar_pcl[:, POINTCLOUD_Z_INDEX] >= configs['lim_z'][0]) & (lidar_pcl[:, POINTCLOUD_Z_INDEX] <= configs['lim_z'][1]))
lidar_box_masked = lidar_box_masked[mask]
print(lidar_box_masked.shape)
print(bev_map[0,0,:,:].shape)

bev_map_cpy = np.zeros((configs['bev_height'], configs['bev_width'], 3))
bev_map_cpy[:, :, 2] = bev_map[0,2,:configs['bev_height'], :configs['bev_width']] / bev_map[0,2,:,:].max() # r_map
bev_map_cpy[:, :, 1] = bev_map[0,1,:configs['bev_height'], :configs['bev_width']] / bev_map[0,1,:,:].max() # g_map
bev_map_cpy[:, :, 0] = bev_map[0,0,:configs['bev_height'], :configs['bev_width']] / bev_map[0,0,:,:].max() # b_map


for box in lidar_box_masked:
    # print("x: ", box[0], " y: ", box[1], " w: ", box[2], " l: ", box[3])
    xyxy = distance_to_pixel(box)
    # print("xy top left: ", xyxy[0], " xy bottom right: ", xyxy[2])
    # cv2.rectangle(bev_map_cpy, (int(xyxy[0][0]), int(xyxy[0][1]), int(xyxy[2][0]), int(xyxy[2][1])), thickness=2, color=(100,0,100))
    # xyxy2 = distance_to_pixel2(box)
    # cv2.rectangle(bev_map_cpy, (int(xyxy2[1]), int(xyxy2[0])), (int(xyxy2[3]), int(xyxy2[2])), thickness=2, color=(100,0,100))

    
    c = ((box[0] - configs['lim_x'][0]) * configs['bev_height'] / (configs['lim_x'][1] - configs['lim_x'][0]), (box[1] - configs['lim_y'][0]) * configs['bev_width'] / (configs['lim_y'][1] - configs['lim_y'][0]))
    rotated = rotate_mtx(xyxy, box[4], (c[1],c[0]))
    cv2.drawContours(bev_map_cpy, [rotated.astype(int)], -1, thickness=2, color=colors[int(box[5])])
    # cv2.putText(bev_map, names[int(cls)], (int(xyxy[0]), int(xyxy[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color=(100,100,0), thickness=2)

tmp = 0
while (1): 
    rotated = rotate_mtx(xyxy, tmp, (c[1],c[0]))
    # cv2.drawContours(bev_map_cpy, [rotated.astype(int)], -1, thickness=2, color=(100,100,0))
    cv2.imshow('bev_map2', bev_map_cpy)
    tmp = tmp + 0.5
    if cv2.waitKey(10) & 0xFF == 27:
        break
cv2.destroyAllWindows

<class 'list'>
[-20.86353529   3.07001304   4.2105113    1.84526304   0.05713253
   1.        ]
(39, 6)
(640, 480)
(640, 480, 3)


In [ ]:
# Get camera

while (1): 
    rotated = rotate_mtx(xyxy, tmp, (c[1],c[0]))
    # cv2.drawContours(bev_map_cpy, [rotated.astype(int)], -1, thickness=2, color=(100,100,0))
    cv2.imshow('camera_img', bev_map_cpy)
    tmp = tmp + 0.5
    if cv2.waitKey(10) & 0xFF == 27:
        break

### Project detections into bev

In [ ]:
# project detected bounding boxes into birds-eye view
def project_detections_into_bev(bev_map, detections, configs, color=[]):
    for row in detections:
        # extract detection
        _id, _x, _y, _z, _h, _w, _l, _yaw = row

        # convert from metric into pixel coordinates
        x = (_y - configs.lim_y[0]) / (configs.lim_y[1] - configs.lim_y[0]) * configs.bev_width
        y = (_x - configs.lim_x[0]) / (configs.lim_x[1] - configs.lim_x[0]) * configs.bev_height
        z = _z - configs.lim_z[0]
        w = _w / (configs.lim_y[1] - configs.lim_y[0]) * configs.bev_width
        l = _l / (configs.lim_x[1] - configs.lim_x[0]) * configs.bev_height
        yaw = -_yaw

        # draw object bounding box into birds-eye view
        if not color:
            color = configs.obj_colors[int(_id)]
        
        # get object corners within bev image
        bev_corners = np.zeros((4, 2), dtype=np.float32)
        cos_yaw = np.cos(yaw)
        sin_yaw = np.sin(yaw)
        bev_corners[0, 0] = x - w / 2 * cos_yaw - l / 2 * sin_yaw # front left
        bev_corners[0, 1] = y - w / 2 * sin_yaw + l / 2 * cos_yaw 
        bev_corners[1, 0] = x - w / 2 * cos_yaw + l / 2 * sin_yaw # rear left
        bev_corners[1, 1] = y - w / 2 * sin_yaw - l / 2 * cos_yaw
        bev_corners[2, 0] = x + w / 2 * cos_yaw + l / 2 * sin_yaw # rear right
        bev_corners[2, 1] = y + w / 2 * sin_yaw - l / 2 * cos_yaw
        bev_corners[3, 0] = x + w / 2 * cos_yaw - l / 2 * sin_yaw # front right
        bev_corners[3, 1] = y + w / 2 * sin_yaw + l / 2 * cos_yaw
        
        # draw object as box
        corners_int = bev_corners.reshape(-1, 1, 2).astype(int)
        cv2.polylines(bev_map, [corners_int], True, color, 2)

        # draw colored line to identify object front
        corners_int = bev_corners.reshape(-1, 2)
        cv2.line(bev_map, (int(corners_int[0, 0]), int(corners_int[0, 1])), (int(corners_int[3, 0]), int(corners_int[3, 1])), (255, 255, 0), 2)


### Add boxes

In [ ]:
def addBoxes(results):
    for result in results:
        boxes = result.boxes
        probs = result.probs
        img = result.orig_img
        classes = boxes.cls.cpu().numpy()
        names = result.names
        # print(boxes, ', prob:')
        for i, xyxy in enumerate(boxes.xyxy.cpu().numpy()):
            # print(xyxy[0])
            cls = classes[i]

            if int(cls) in colors_map:
                color = colors_map[cls]
                cv2.rectangle(img, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), thickness=2, color=color)
                cv2.putText(img, names[int(cls)], (int(xyxy[0]), int(xyxy[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color=color, thickness=2)
            
    cv2.imshow('Camera and detected objects', img)
    cv2.waitKey(0) 
    return img

In [ ]:
# Utility for fixing 
range_image = v2.perception.lidar.RangeImage(lidar.range_image_return1.values[0], lidar.range_image_return1.shape[0])

print(range_image.values)
print(type(range_image))
print(type(range_image_orig))
rit = range_image.tensor
print(rit)
print(type(rit))
# tf.convert_to_tensor(lidar_calibration.extrinsic.transform.tolist())

points = v2.perception.utils.lidar_utils.convert_range_image_to_point_cloud(range_image, lc2, keep_polar_features=True)

In [6]:
row

key.segment_context_name                                                             10023947602400723454_1120_000_1140_000
key.frame_timestamp_micros                                                                                 1552440195362591
key.laser_name                                                                                                          [1]
[LiDARComponent].range_image_return1.values                               [[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...
[LiDARComponent].range_image_return1.shape                                                                  [[64, 2650, 4]]
[LiDARComponent].range_image_return2.values                               [[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...
[LiDARComponent].range_image_return2.shape                                                                  [[64, 2650, 4]]
key.camera_name                                                           [nan, nan, nan, nan, 1.0, nan, 1.0, nan, nan, ...
key.came